# Event detection from Wikipedia pages

In [1]:
!pip install pandarallel sparql-client p_tqdm

     |████████████████████████████████| 225kB 10.4MB/s 
     |████████████████████████████████| 194kB 14.3MB/s 
     |████████████████████████████████| 225kB 14.3MB/s 
     |████████████████████████████████| 51kB 7.7MB/s 
     |████████████████████████████████| 112kB 18.7MB/s 
     |████████████████████████████████| 133kB 16.0MB/s 
  Created wheel for pandarallel: filename=pandarallel-1.5.1-cp36-none-any.whl size=17126 sha256=b662332e9bac2692fa489543c9abbb4e7755a8104fb0d83ce3c85687099c57e0
  Stored in directory: /root/.cache/pip/wheels/6f/c8/e5/d43fa63105ce1dd22f4df51bc2edfefd54d92ce64f25326314
  Created wheel for sparql-client: filename=sparql_client-3.8-cp36-none-any.whl size=13666 sha256=6403c6d9ec4302a84ee6dc04d7fc839f10682683c6dc5dc885bf8ef3c1b19f5a
  Stored in directory: /root/.cache/pip/wheels/37/c6/b3/dc96bcb728d766c48ac7771591c063edd598ecaf13ec30552c
  Created wheel for p-tqdm: filename=p_tqdm-1.3.3-cp36-none-any.whl size=3988 sha256=f3aa78cad495004c15f2584af0c68570bfdcb7a6c65

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/mhc/')

In [4]:
%cd "/content/drive/My Drive/Colab Notebooks/mhc/"

/content/drive/My Drive/Colab Notebooks/mhc


In [5]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [1]:
import pandas as pd
import numpy as np

from pathlib import Path
from collections import Counter

import tensorflow as tf

from text import PreProcessing
from buildGloveVocab import GloveVocab

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from tqdm import tqdm
import multiprocessing as mp
from multiprocessing import Pool

from p_tqdm import p_map

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## Download Wikipedia pages

In [ ]:
from wikiapi import WikiWrapper

In [ ]:
q_hist = (
    """
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT DISTINCT ?Event_1
    WHERE { ?Event_1 a dbo:Event .
            ?Event_1 a dbo:MilitaryConflict . }

    LIMIT 1000
    """
)

q_nonhist_1 = (
    """
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT DISTINCT ?Artist_1
    WHERE { ?Artist_1 a dbo:Artist . }

    LIMIT 500
    """
)

q_nonhist_2 = (
    """
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT DISTINCT ?Animal_1
    WHERE { ?Animal_1 a dbo:Animal . }

    LIMIT 500
    """
)

In [ ]:
hist_events_name = WikiWrapper.download_pages_name(q_hist)
non_hist_events_name = WikiWrapper.download_pages_name(q_nonhist_1)
non_hist_events_name = non_hist_events_name + WikiWrapper.download_pages_name(q_nonhist_2)

In [ ]:
df = pd.DataFrame(hist_events_name, columns=["Name"])
df["Abstract"] = ""
df["Label"] = 1

In [ ]:
df1 = pd.DataFrame(non_hist_events_name, columns=["Name"])
df1["Abstract"] = ""
df1["Label"] = 0

In [ ]:
df = pd.concat([df, df1])
df.reset_index(inplace=True, drop=True)
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df["Abstract"] = df.parallel_apply(WikiWrapper.get_extract, axis=1)

In [ ]:
df.dropna(subset=["Abstract"], inplace=True)
df.reset_index(inplace=True, drop=True)

In [ ]:
df["Abstract"] = df["Abstract"].parallel_apply(lambda x: x.replace(",", "").replace("|", ""))

In [ ]:
df.to_csv("data/wiki/wiki.csv", index=False, sep="|")

## Classification

In [2]:
df = pd.read_csv("data/wiki/wiki.csv", sep="|")

In [8]:
df.Abstract = df.Abstract.parallel_apply(lambda x: PreProcessing.cleanText(x))

In [9]:
df.drop(columns=["Name"], inplace=True)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df.Abstract.values, df.Label.values, test_size=0.2, random_state=42, stratify=df.Label.values)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

In [11]:
BATCH_SIZE = 64
EPOCHS = 20

In [12]:
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(X_train).batch(BATCH_SIZE)
vectorizer.adapt(text_ds)

In [13]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

Skip the two cells below if you have already a compressed representation of the embedding matrix

In [ ]:
path_to_glove_file = Path("wordemb/glove.840B.300d.txt")

embeddings_index = {}
with open(path_to_glove_file) as f:
    for i, line in enumerate(f):
        if i % 100000 == 0:
            print('- At line {}'.format(i))

        line = line.strip().split()

        if len(line) != 300 + 1:
            continue

        word = line[0]
        coefs = " ".join(line[1:])
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

- At line 0
- At line 100000
- At line 200000
- At line 300000
- At line 400000
- At line 500000
- At line 600000
- At line 700000
- At line 800000
- At line 900000
- At line 1000000
- At line 1100000
- At line 1200000
- At line 1300000
- At line 1400000
- At line 1500000
- At line 1600000
- At line 1700000
- At line 1800000
- At line 1900000
- At line 2000000
- At line 2100000
Found 2195876 word vectors.


In [ ]:
num_tokens = len(voc) + 2
embedding_dim = 300
hits = 0
misses = 0

# Prepare embedding matrix
embeddings = np.zeros((num_tokens, embedding_dim))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embeddings[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

np.savez_compressed("data/wiki/glove_wiki.npz", embeddings=embeddings)

100%|██████████| 20000/20000 [00:00<00:00, 216934.23it/s]
Converted 15502 words (4498 misses)


Reload your embedding matrix here

In [14]:
embeddings = np.load("data/wiki/glove_wiki.npz")['embeddings']

In [15]:
X_train = vectorizer(np.array([[s] for s in X_train])).numpy()
X_val = vectorizer(np.array([[s] for s in X_val])).numpy()
X_test = vectorizer(np.array([[s] for s in X_test])).numpy()

In [19]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(None,), dtype="int64"),
    tf.keras.layers.Embedding(len(voc) + 2, 300, embeddings_initializer=tf.keras.initializers.Constant(embeddings), trainable=False),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 300)         6000600   
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 256)         439296    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 256)               394240    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 6,867,161
Trainable params: 866,561
Non-trainable params: 6,000,600
______________________________________

In [21]:
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(X_val, y_val))

Epoch 1/20
18/18 [==============================] - 2s 129ms/step - loss: 0.2249 - accuracy: 0.8900 - val_loss: 0.0973 - val_accuracy: 0.9652
Epoch 2/20
18/18 [==============================] - 1s 56ms/step - loss: 0.0835 - accuracy: 0.9467 - val_loss: 0.0536 - val_accuracy: 0.9512
Epoch 3/20
18/18 [==============================] - 1s 57ms/step - loss: 0.0518 - accuracy: 0.9581 - val_loss: 0.0498 - val_accuracy: 0.9721
Epoch 4/20
18/18 [==============================] - 1s 56ms/step - loss: 0.0523 - accuracy: 0.9747 - val_loss: 0.0495 - val_accuracy: 0.9721
Epoch 5/20
18/18 [==============================] - 1s 56ms/step - loss: 0.0468 - accuracy: 0.9668 - val_loss: 0.0455 - val_accuracy: 0.9756
Epoch 6/20
18/18 [==============================] - 1s 57ms/step - loss: 0.0472 - accuracy: 0.9712 - val_loss: 0.0497 - val_accuracy: 0.9721
Epoch 7/20
18/18 [==============================] - 1s 57ms/step - loss: 0.0459 - accuracy: 0.9616 - val_loss: 0.0507 - val_accuracy: 0.9512
Epoch 8/20
1

In [22]:
_, test_acc = model.evaluate(X_test, y_test)

print('Test Accuracy: {}'.format(test_acc))

12/12 [==============================] - 0s 16ms/step - loss: 0.0590 - accuracy: 0.9638
Test Accuracy: 0.9637883305549622
